## Logisk imputering
<font size = 2>Logisk imputering er å tilordne en variabel en verdi utledet fra logiske regler:
- Hvis betingelse er sann, så skal verdi A settes
- Hvis  betingelse er sann, så skal verdi A settes, hvis ikke, så skal verdi B settes

### Funksjoner for logisk imputering
<font size=2>Det er blitt laget tre funksjonene i Python som henholdsvis endrer en negativ verdi til tallet 0, en negativ verdi til positiv verdi, og en missingverdi til tallet 0. Selve koden finnes i neste paragraf og eksempler på hvordan de brukes finnes lenger ned i noten:
<li> <code>set_neg_to_zero(df, col_names)</code>:
<li> <code>set_neg_to_plus(df, col_names)</code>:
<li> <code>set_miss_to_zero(df, col_names)</code>:
<ul>
</font>

In [ ]:
def set_neg_to_zero(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: 0 if x<0 else x)
    return df

def set_neg_to_plus(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: abs(x) if x<0 else x)
    return df
    
def set_miss_to_zero(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: 0 if pd.isna(x) else x)
    return df

#### Importere biblioteker og sette displayopsjoner

In [ ]:
import dapla as dp
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 5000
pd.options.display.float_format = "{:.2f}".format

#### Leser inn eksempeldataene

In [ ]:
df_areal = dp.read_pandas("/felles/veiledning/pyspark/eksempler/areal")
df_bnp = dp.read_pandas("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall_2020 = dp.read_pandas("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
df_innbyggerantall_2018 = dp.read_pandas("/felles/veiledning/pyspark/eksempler/innbyggerantall/2018")

#### Koble innbyggerantall på arealdatasett (koblingsnøkkel er Landkode) til bruk for å illustrere logisk imputering (kode)

In [ ]:
df_areal_innbyggerantall_20 = pd.merge(df_areal, df_innbyggerantall_2020, how='left', on='Landkode')

#viser hvilke land med areal (venstre sett) som ikke har registrert innbyggerantall 2020
df_areal_innbyggerantall_20.loc[pd.isna(df_areal_innbyggerantall_20.Innbyggerantall)]

#### Erstatte missing-verdier i innbyggertall ved bruk av funksjonen set_miss_to_zero (kode)

In [ ]:
# kaller på funksjonen set_miss_to_zero
df_areal_innbyggerantall_20 = set_miss_to_zero(df_areal_innbyggerantall_20,col_names={'Innbyggerantall','År','Kilde_y','Land_y'})

# viser resultat av funksjonen
df_areal_innbyggerantall_20.loc[pd.isna(df_areal_innbyggerantall_20.Innbyggerantall)]

#### Erstatte negativ verdi i bnp med 0 ved bruk av funksjonen set_neg_to_zero (kode)

In [ ]:
# Rydder først ut missing-verdier
df_bnp.drop(df_bnp.loc[pd.isna(df_bnp.BNP)].index, axis=0, inplace=True)

# gjør om BNP til integer
df_bnp.BNP = df_bnp.BNP.astype('Int64')

# skriver ut observasjoner hvor BNP<0
df_bnp.loc[df_bnp.BNP<0]

In [ ]:
# kaller på funksjonen set_neg_to_zero
df_bnp_imp_zero = set_neg_to_zero(df_bnp.copy(),['BNP'])

# viser resultatet av funksjonen
df_bnp_imp_zero.loc[df_bnp.Landkode.isin(['PT','SI'])]

In [ ]:
# Ser at opprinnelig datasett har fortsatt opprinnelig verdi
df_bnp.loc[df_bnp.Landkode.isin(['PT','SI'])]

#### Erstatte negativ verdi med absoluttverdien ved bruk av funksjonen set_neg_to_plus (kode)

In [ ]:
# kaller på funksjonen set_neg_to_plus
df_bnp_imp_plus = set_neg_to_plus(df_bnp.copy(),['BNP'])

# viser resultatet av funksjonen
df_bnp_imp_plus.loc[df_bnp.Landkode.isin(['PT','SI'])]